# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,9.89,71,31,10.26,ZA,1727655759
1,1,stanley,54.8680,-1.6985,10.16,84,20,3.09,GB,1727655761
2,2,wiwili,13.6268,-85.8254,24.59,84,64,1.02,NI,1727655762
3,3,anadyr,64.7500,177.4833,5.28,86,75,5.00,RU,1727655764
4,4,port hueneme,34.1478,-119.1951,25.23,72,0,5.14,US,1727655103


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat",
                                      geo= True, 
                                      tiles = "OSM",
                                      size = "Humidity",
                                      frame_width = 800,
                                      frame_height = 600,
                                      scale = 0.5,
                                      color = "City")
                                

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# A max_temp lower than 27 but higher than 21
min_temp = city_data_df["Max Temp"]>21
max_temp = city_data_df["Max Temp"]<27

#Wind speed less than 4.5m/s
wind_speed = city_data_df["Wind Speed"]<4.5

#Zero cloudiness
zero_cloudiness = city_data_df["Cloudiness"]==0

# Narrow down cities that fit criteria and drop any results with null values
city_narrow_data_df = city_data_df[min_temp & max_temp & wind_speed & zero_cloudiness]
city_narrow_data_df

# Drop any rows with null values
clean_city_data_df = city_narrow_data_df.dropna()

# Display sample data
clean_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
150,150,shenjiamen,29.9576,122.2980,25.76,87,0,4.02,CN,1727656011
161,161,chimoio,-19.1164,33.4833,23.92,46,0,1.15,MZ,1727656029
195,195,yorkton,51.2167,-102.4677,23.91,46,0,4.12,CA,1727656086
235,235,miandrivazo,-19.5167,45.4667,25.03,63,0,0.68,MG,1727656151
311,311,grand baie,-20.0182,57.5802,22.47,76,0,0.89,MU,1727656286
322,322,at taj,24.2049,23.2857,23.75,35,0,3.79,LY,1727656305
374,374,brookings,44.3114,-96.7984,24.03,47,0,4.47,US,1727656255
378,378,tolanaro,-25.0319,46.9987,23.04,83,0,3.09,MG,1727656398
397,397,crane,31.3974,-102.3501,26.58,25,0,4.19,US,1727656426
428,428,phalia,32.4297,73.5775,25.74,66,0,0.95,PK,1727656483


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
150,150,shenjiamen,29.9576,122.2980,25.76,87,0,4.02,CN,1727656011,
161,161,chimoio,-19.1164,33.4833,23.92,46,0,1.15,MZ,1727656029,
195,195,yorkton,51.2167,-102.4677,23.91,46,0,4.12,CA,1727656086,
235,235,miandrivazo,-19.5167,45.4667,25.03,63,0,0.68,MG,1727656151,
311,311,grand baie,-20.0182,57.5802,22.47,76,0,0.89,MU,1727656286,
322,322,at taj,24.2049,23.2857,23.75,35,0,3.79,LY,1727656305,
374,374,brookings,44.3114,-96.7984,24.03,47,0,4.47,US,1727656255,
378,378,tolanaro,-25.0319,46.9987,23.04,83,0,3.09,MG,1727656398,
397,397,crane,31.3974,-102.3501,26.58,25,0,4.19,US,1727656426,
428,428,phalia,32.4297,73.5775,25.74,66,0,0.95,PK,1727656483,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
    "categories": categories,
    "apiKey": "d5a2d3abf34648f7a2611c5fc6f9c924"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shenjiamen - nearest hotel: Zhonghan Hotel
chimoio - nearest hotel: Hotel Amirana
yorkton - nearest hotel: Holiday Inn Express
miandrivazo - nearest hotel: Baobab
grand baie - nearest hotel: Safari
at taj - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
tolanaro - nearest hotel: Hôtel Mahavokey
crane - nearest hotel: No hotel found
phalia - nearest hotel: No hotel found
al hillah - nearest hotel: فندق الاسراء السياحي
waddan - nearest hotel: No hotel found
marrakesh - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
sao joao da barra - nearest hotel: Pousada Mediterrâneo


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
150,150,shenjiamen,29.9576,122.2980,25.76,87,0,4.02,CN,1727656011,Zhonghan Hotel
161,161,chimoio,-19.1164,33.4833,23.92,46,0,1.15,MZ,1727656029,Hotel Amirana
195,195,yorkton,51.2167,-102.4677,23.91,46,0,4.12,CA,1727656086,Holiday Inn Express
235,235,miandrivazo,-19.5167,45.4667,25.03,63,0,0.68,MG,1727656151,Baobab
311,311,grand baie,-20.0182,57.5802,22.47,76,0,0.89,MU,1727656286,Safari
322,322,at taj,24.2049,23.2857,23.75,35,0,3.79,LY,1727656305,No hotel found
374,374,brookings,44.3114,-96.7984,24.03,47,0,4.47,US,1727656255,Quality Inn
378,378,tolanaro,-25.0319,46.9987,23.04,83,0,3.09,MG,1727656398,Hôtel Mahavokey
397,397,crane,31.3974,-102.3501,26.58,25,0,4.19,US,1727656426,No hotel found
428,428,phalia,32.4297,73.5775,25.74,66,0,0.95,PK,1727656483,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng","Lat",
                                    geo = True,
                                    tiles = "OSM",
                                    frame_width = 800,
                                    frame_height = 600,
                                    size = "Humidity",
                                    color = "City",
                                    hover_cols = ["City","Hotel Name","Country"]
                                   )
                                   

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)